In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
faq = pd.read_excel("../data/01_База_знаний.xlsx")
faq.head()

,Тема,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Что нельзя публиковать на RUTUBE,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
2,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за искус...,Монетизация на RUTUBE зависит в том числе от к...,МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Почему могут отключить монетизацию на видео и ...,"Для каких статусов доступна монетизация, и поч...","Монетизацию на RUTUBE можно подключить, если в...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,Авторское право,Какой контент можно использовать для монетизац...,"То, что вы создали сами: видео, которое вы сня...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


In [3]:
faq = faq.drop(['Тема', 'Классификатор 1 уровня', 'Классификатор 2 уровня'], axis=1)
faq.head()

,Вопрос из БЗ,Ответ из БЗ
0,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...
1,Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к..."
2,Почему могут отключить монетизацию из-за искус...,Монетизация на RUTUBE зависит в том числе от к...
3,"Для каких статусов доступна монетизация, и поч...","Монетизацию на RUTUBE можно подключить, если в..."
4,Какой контент можно использовать для монетизац...,"То, что вы создали сами: видео, которое вы сня..."


In [4]:
faq.to_csv('../data/knowledge_base.csv', index=False)

In [5]:
import torch

torch.cuda.is_available()

True

In [6]:
from langchain_community.document_loaders import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [7]:
file_path = '../data/knowledge_base.csv'

embeddings = HuggingFaceEmbeddings(model_name="deepvk/USER-bge-m3", cache_folder='./cache_data/embeddings/')

In [8]:
raw_documents = CSVLoader(file_path, encoding='utf-8').load()

db = Chroma.from_documents(raw_documents, embeddings)

OutOfMemoryError: CUDA out of memory. Tried to allocate 122.00 MiB. GPU 0 has a total capacity of 2.00 GiB of which 0 bytes is free. Of the allocated memory 1.34 GiB is allocated by PyTorch, and 21.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
raw_documents

In [ ]:
test_retriever = db.as_retriever(
    # search_type='mmr',
    search_type='similarity',
    search_kwargs={
        'k': 3,
        # 'fetch_k': 5,
    }
)

In [ ]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

cross_encoder3 = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3", model_kwargs={'cache_dir': './data/cross_encoders'})

In [ ]:
from langchain.retrievers.document_compressors import CrossEncoderReranker

compressor3 = CrossEncoderReranker(model=cross_encoder3, top_n=3)

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever, EnsembleRetriever

compression_retriever3 = ContextualCompressionRetriever(
    base_compressor=compressor3,
    base_retriever=test_retriever,
)

In [ ]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(raw_documents, k=3)

In [ ]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[compression_retriever3, bm25_retriever],
    weights=[0.7, 0.3],
)

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import GigaChat

template = """Ты интеллектуальный помощьник компании rutube и помогаешь отвечать на вопросы. Ответь на вопрос используя следующий контекст:

{context}

Вопрос: {question}

Если не можешь найти ответ в контексте, вежливо скажи, что не знаешь ответ
"""
prompt = ChatPromptTemplate.from_template(template)
model = GigaChat(credentials="Yjg5N2U0ZWItNzI2YS00OTg1LTk0ZTQtNWMwM2IwYjBlYWZhOjk4MzRkZjhjLTE2MmItNGE5ZS05MjQ0LWE1YWQ0MDNlYjdlMQ==", verify_ssl_certs=False)

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

for a in chain.stream("У контента из интернета есть авторские права?"):
    print(a, end = '')


ModuleNotFoundError: No module named 'langchain_core'

In [ ]:
def context_precision(predictions, ground_truths):
    precision_scores = []
    for pred, truth in zip(predictions, ground_truths):
        relevant_items = set(truth)
        retrieved_items = set(pred)
        if retrieved_items:
            precision_scores.append(len(relevant_items & retrieved_items) / len(retrieved_items))
        else:
            precision_scores.append(0)
    return sum(precision_scores) / len(precision_scores)

def faithfulness(predictions, ground_truths):
    return sum(1 for pred, truth in zip(predictions, ground_truths) if pred == truth) / len(predictions)

def answer_relevancy(predictions, ground_truths):
    relevancy_scores = []
    for pred, truth in zip(predictions, ground_truths):
        relevancy_scores.append(len(set(pred) & set(truth)) / len(set(truth)) if truth else 0)
    return sum(relevancy_scores) / len(relevancy_scores)

In [10]:
data = pd.read_excel("../data/02_Реальные_кейсы.xlsx")
data.head()

,Вопрос пользователя,Ответ сотрудника,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Что запрещено в монетизации и что можно выклад...,"Здравствуйте!\nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Для монетизации можно использовать то, что вы ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


In [11]:
data = data.drop(['Вопрос из БЗ', 'Ответ из БЗ', 'Классификатор 1 уровня', 'Классификатор 2 уровня'], axis=1)
data.head()

,Вопрос пользователя,Ответ сотрудника
0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...
1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ..."
3,Что запрещено в монетизации и что можно выклад...,"Здравствуйте!\nМонетизация может отключиться, ..."
4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Для монетизации можно использовать то, что вы ..."


In [12]:
data.to_csv('../data/test_base.csv', index=False)

In [14]:
data = pd.read_csv('../data/test_base.csv', encoding='utf-8')

In [ ]:
predictions = []
for question in data['Вопрос пользователя']:
    prediction = chain.invoke(question)
    predictions.append(prediction)

precision = context_precision(predictions, data['Ответ сотрудника'])
faithfulness_score = faithfulness(predictions, data['Ответ сотрудника'])
relevancy = answer_relevancy(predictions, data['Ответ сотрудника'])

result = pd.DataFrame({
    'Metric': ['Context Precision', 'Faithfulness', 'Answer Relevancy'],
    'Score': [precision, faithfulness_score, relevancy]
})

print(result)